In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv(r'C:\Users\ali.kilinc\Desktop\NEW DATA\INPUT\M1_PYTHON_INPUT.csv', index_col= False)
df = df.drop(df.columns[[0]], axis=1)
df.replace('?', np.NaN)

#print(df.columns[0])
#print(df.head(10))
#print(df.columns)

df = pd.DataFrame(df)

X = df.iloc[:, :-1].values
xcol = df.columns[:-1]
Y = df.iloc[:, 59:].values
ycol = df.columns[59:]

df_x = pd.DataFrame(X, columns = xcol)
df_y = pd.DataFrame(Y, columns = ycol)
#print(ycol)
#print(xcol)

In [2]:
for col in ['SERVICE_GROUP_CODE', 'SERVICE_TYPE_CODE', 'FULL_TRUCK_TYPE','SYSTEM_FULL_TRUCK_TYPE',
        'DT_RDY_WEEK', 'DT_RDY_DAY', 'CUSTOMER_COMPANY_CODE','LOADING_COMPANY_CODE',
       'CUSTOMS_COMPANY_CODE', 'CUSTOMSEE_COMPANY_CODE','DELIVERY_COMPANY_CODE', 'CUSTOMS_TYPE', 'CARRY_TYPE', 'SPECIAL_LOAD_TYPE','FORBIDDEN_MODE',
       'GATE_COMPANY_CODE','LOADING_COUNTRY', 'DELIVERY_COUNTRY', 'DT_ETA_WEEK','DT_ETA_DAY']:
    df_x[col] = df_x[col].astype('object')
       
for col in ['PW', 'WH_DELIVERED', 'GROSS_WEIGHT', 'FREEZONE', 'REVENUE']:
    df_x[col] = df_x[col].astype('int')
    
df_x.info()
df_x.dtypes
df_x.isnull().sum()

num_cols = list(df_x._get_numeric_data().columns)
#print(num_cols)

fix_cols = []
for col in df_x.columns:
    if df_x[col].nunique() == 1:
        fix_cols.append(col)
    else:
        pass

emp_cols = []
for col in df_x.columns:
    if df_x[col].count() > 0:
        pass
    else:
        emp_cols.append(col)

cat_cols = list(set(df_x.columns) - set(num_cols) - set(emp_cols) - set(fix_cols))
num_cols = list(set(num_cols) - set(fix_cols) - set(emp_cols))
#print(df_x[cat_cols])
#print(df_x['GATE_COMPANY_CODE'].value_counts())
#df_x['CUSTOMSEE_COMPANY_CODE'].value_counts()
#customsee float geldiği için .0 var stringinde, sorun yok ama. 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 422 entries, 0 to 421
Data columns (total 59 columns):
RUN_ID                     0 non-null object
ACTIVE                     0 non-null object
ORGANIZATION_CODE          0 non-null object
SERVICE_GROUP_CODE         422 non-null object
ORDER_STATE                0 non-null object
SERVICE_TYPE_CODE          422 non-null object
PW                         422 non-null int32
FULL_TRUCK_TYPE            422 non-null object
RELATED_ORDER_CODE         0 non-null object
RELATED_ORDER_TYPE         0 non-null object
DATE_BOOKING               0 non-null object
DATE_LOADING               0 non-null object
WH_DELIVERED               422 non-null int32
DATE_WH_ENTRY              0 non-null object
LAST_WH                    0 non-null object
DATE_LAST_WH_FINISH        0 non-null object
CUSTOMER_COMPANY_CODE      422 non-null object
LOADING_COMPANY_CODE       422 non-null object
CUSTOMS_COMPANY_CODE       422 non-null object
CUSTOMSEE_COMPANY_CODE    

In [3]:
from sklearn import preprocessing

scaler = preprocessing.MinMaxScaler(copy=True, feature_range=(0, 1))
le = preprocessing.LabelEncoder()
ohe = preprocessing.OneHotEncoder(categories = 'auto')

df_x_new = pd.DataFrame()
map =[]
i = 0

for col in cat_cols:
    
    df_x[col] = le.fit_transform(df_x[col].astype(str))
    mappers = dict(zip(le.classes_, le.transform(le.classes_))).copy()
    map_list = list(mappers.keys())
    #inv_mappers = {v: k for k, v in mappers.items()} KEY ile VALUE Yerini değiştiriyor
    #inv_mappers = {int(k):str(v) for k,v in inv_mappers.items()} Key ile value tipini değiştiriyor
    map.append(dict(zip(le.classes_, le.transform(le.classes_))).copy())
      
    if df_x[col].nunique()>1:
        df_r = ohe.fit_transform(df_x[[col]]).toarray()
        df_r_df = pd.DataFrame(df_r, columns = str(i)+'_'+le.classes_)
        #df_r_df = pd.DataFrame(df_r, columns = [str(i)+'_'+le.classes_]) yaptığımda multiindex haline getirdi sütunları
        #df_r_df.columns = df_r_df.columns.astype(str)
        df_x_new = pd.concat([df_x_new, df_r_df], axis=1)
        #print(le.classes_)
        #print(le.transform(le.classes_))
    else:
        
        df_r_df = pd.DataFrame(df_x[col], columns = str(i)+'_'+le.classes_)
        #df_r_df = pd.DataFrame(df_x[col])
        df_x_new = pd.concat([df_x_new, df_r_df], axis=1)
    #print(np.isnan(df_r_df).any()) check null items
    #print(np.isfinite(df_r_df).any()) check infinite items
    i = i+1
#!!!!! eğer (XYZ,) şeklinde bir sütun başlığı oluşturuyorsa multiindex yapıyor demektir.
#Bu durumda dataframe içinde columns = [] olmalı columns = ... yerine veya tam tersi

min_max = {}
for col in num_cols:
    min_col = 0
    max_col = 0
    df_x[col] = pd.to_numeric(df_x[col])
    min_col = df_x[col].min()
    max_col = df_x[col].max()
    min_max[str(i)+'_'+col] = [min_col,max_col]
    #print(min_max)
    df_s = scaler.fit_transform(df_x[[col]])
    df_s_df = pd.DataFrame(df_s, columns = [str(i)+'_'+col])
    df_s_df.columns = df_s_df.columns.astype(str)
    df_x_new = pd.concat([df_x_new, df_s_df], axis=1)
    i=i+1
    

full_data = pd.concat([df_x_new, df_y], axis = 1)

C:\ProgramData\Anaconda3\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int32 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int32 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int32 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [5]:
from sklearn.model_selection import train_test_split

x = np.array(full_data.drop('classification', axis=1))
y = np.array(full_data['classification'])
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state = 3)

from sklearn import neighbors
from sklearn.metrics import mean_squared_error 
from math import sqrt
import matplotlib.pyplot as plt
"""
rmse_val = [] #to store rmse values for different k
for K in range(50):
    K = K+1
    
    model = neighbors.KNeighborsRegressor(n_neighbors = K)
    model.fit(x_train, y_train)  #fit the model
    pred=model.predict(x_test) #make prediction on test set
    error = sqrt(mean_squared_error(y_test,pred)) #calculate rmse
    rmse_val.append(error) #store rmse values
    print('RMSE value for k= ' , K , 'is:', error)

curve = pd.DataFrame(rmse_val) #elbow curve 
curve.plot()
"""

"\nrmse_val = [] #to store rmse values for different k\nfor K in range(50):\n    K = K+1\n    \n    model = neighbors.KNeighborsRegressor(n_neighbors = K)\n    model.fit(x_train, y_train)  #fit the model\n    pred=model.predict(x_test) #make prediction on test set\n    error = sqrt(mean_squared_error(y_test,pred)) #calculate rmse\n    rmse_val.append(error) #store rmse values\n    print('RMSE value for k= ' , K , 'is:', error)\n\ncurve = pd.DataFrame(rmse_val) #elbow curve \ncurve.plot()\n"

In [6]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(k_neighbors = 2)
df_x_new_res, df_y_res = sm.fit_resample(df_x_new, np.ravel(df_y))

df_x_new_res = pd.DataFrame(df_x_new_res, columns = df_x_new.columns)
df_y_res = pd.DataFrame(df_y_res, columns = df_y.columns)

col_prefix=[]
for col in df_x_new.columns:
    #print(col[0:20])
    if col.split("_")[0] in col_prefix:
        pass
    else:
        col_prefix.append(col.split("_")[0])

#print(df_x_new_res.columns)
#print(df_y_res.columns)

df_result = pd.DataFrame()
cat = 0
num = 0
for item in col_prefix:
    #print(df_x_new_res[[col for col in df_x_new_res if col.startswith(item+'_')]].head(4))
    df_reverse = pd.DataFrame()
    df_reverse = df_x_new_res[[col for col in df_x_new_res if col.startswith(item+'_')]]
    if len([col for col in df_x_new_res if col.startswith(item+'_')])>1:
        df_reverse = df_reverse.idxmax(1)
        df_reverse = pd.DataFrame(df_reverse,columns = [cat_cols[cat]])
        df_reverse = df_reverse.applymap(lambda x: x.replace(item+'_', ''))
        #df_reverse.columns = cat_cols[cat]
        df_result = pd.concat([df_result, df_reverse], axis = 1)
        #print(df_reverse.head(4))
        #print(cat_cols[cat])
        cat = cat+1
    else:
        for col in df_reverse:
            #print(min_max[col][:])
            mn = np.array(min_max[col][:1])
            mx = np.array(min_max[col][1:])
            df_reverse[col] = round(df_reverse[col]*(mx-mn)+mn,0)
            df_result = pd.concat([df_result, df_reverse], axis = 1)
    #print(df_result.head(4))
#df_result.columns = pd.concat([pd.Series(cat_cols), pd.Series(num_cols)], axis=1)       

for col in (fix_cols + emp_cols):
    df_result = pd.concat([df_result, df_x[col].iloc[:len(df_y_res)]],axis = 1)
    df_result[col] = df_result[col].ffill()
    #571 elemnalı input matrisinden çektiğim için kısa kaldı sütun ve eksik elemanlar NULL olarak geldi. Onları doldurmuş olduk.
    
df_result = pd.concat([df_result, df_y_res], axis=1)

print(df_result)
print(df_y_res['classification'].value_counts())
print(df_y['classification'].value_counts())


#writer = pd.ExcelWriter(r'C:\Users\ali.kilinc\Desktop\NEW DATA\OUTPUT\N29_PYTHON_OUTPUT.xlsx')
#df_result.to_excel(writer,'OUTPUT')
#writer.save()


C:\ProgramData\Anaconda3\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


    DT_RDY_DAY DT_ETA_WEEK DT_RDY_WEEK SERVICE_TYPE_CODE DT_ETA_DAY  \
0            5          42          40            Normal          1   
1            5          42          40            Normal          1   
2            5          42          40            Normal          1   
3            5          42          40            Normal          1   
4            5          41          40         Süper Exp          2   
5            5          41          40         Süper Exp          2   
6            5          42          40            Normal          2   
7            5          42          40            Normal          2   
8            5          42          40            Normal          1   
9            5          42          40            Normal          1   
10           5          42          40            Normal          1   
11           5          42          40            Normal          2   
12           5          42          40            Normal          1   
13    